<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=84c73f24d4068cbddc48755cb69d5e4749da8e61941e59ae21204a1adc661fe7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-07 16:26:07
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.58 C
-------------------
Today PnL: 43.53 K (0.28%)
Current PnL: -24.79 L (-15.81%)
CY Booked + Current PnL: -10.50 L (-6.69%)
-------------------
Total profit:  2.35 L
Total loss:  -27.13 L
-------------------
Total Booked + Current PnL: 16.48 L (12.7%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.02%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.52 C
Est FTT PnL: 93.80 L (59.23%)
Deployed:  1.30 C
Current:  1.58 C
CAGR/XIRR %: 11.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.39,15.91,4.73,21.40,14552.0,42237.0,307663.0,7.60,68.0,X-LC,5.24,14.0,2.90,1.94,23.25,XY25,NTT,FMCG
49,MASFIN,398.61,-0.87,2.62,18.93,22.05,19033.0,2565.0,100545.0,-11.47,61.0,H-SC,6.92,168.0,0.13,0.63,46.15,XR,ATH,FINANCE
17,COALINDIA,484.83,1.08,6.15,12.32,19.23,19676.0,9250.0,159710.0,24.27,84.0,L-LC,6.31,182.0,0.47,1.01,28.73,XY25,ATH,MINING
80,TTKPRESTIG,770.00,-1.17,-20.60,26.06,0.09,20852.0,-20762.0,80015.0,85.70,38.0,M-SC,3.20,253.0,-1.00,0.51,5.12,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.64,-13.20,15.23,0.02,23076.0,-23043.0,151516.0,131.03,58.0,M-SC,7.51,240.0,-1.00,0.96,31.79,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAELXSI,9161.00,9.50,-15.65,56.52,32.03,62854.0,-20628.0,111207.0,-8.65,73.0,H-SC,6.99,157.0,-0.33,0.70,24.56,OX40N,NTT,IT
13,BSOFT,831.70,5.25,-19.61,87.57,50.79,99013.0,-27586.0,113067.0,3.21,59.0,H-SC,10.66,171.0,-0.28,0.71,31.65,XR,ATH,IT
23,DMART,5391.45,4.86,-7.65,40.34,29.60,92982.0,-19104.0,230496.0,-21.24,52.0,X-LC,9.64,10.0,-0.21,1.46,12.86,X40N,ATH,FMCG
37,IEX,219.00,4.11,5.04,41.49,48.63,87144.0,10082.0,210036.0,-30.08,71.0,H-SC,13.83,137.0,0.12,1.33,17.25,XR,NTT,MISC
44,JCHAC,2282.00,2.71,-35.32,54.60,-0.01,45133.0,-45143.0,82662.0,16813.04,47.0,M-SC,14.15,235.0,-1.00,0.52,5.80,OX40N,BTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,-3.45,-26.81,90.75,39.61,68081.0,-27480.0,75020.0,-50.46,62.0,H-SC,31.61,136.0,-0.40,0.47,21.43,AR,ATH,MISC
35,ICICIGI,2252.93,-2.18,4.15,14.58,19.34,30387.0,8308.0,208417.0,-17.91,50.0,X-MC,3.81,71.0,0.27,1.32,20.75,X40,ATH,INSURANCE
40,INDUSINDBK,1800.00,-1.79,-33.71,100.48,32.89,51423.0,-26030.0,51177.0,-732.12,64.0,L-MC,7.46,259.0,-0.51,0.32,40.94,XR,NTT,BANKS
53,PGHH,17907.65,-1.68,-4.85,41.03,34.19,83360.0,-10352.0,203168.0,-33.05,41.0,X-MC,4.06,60.0,-0.12,1.28,1.08,X40,ATH,FMCG
71,STARHEALTH,761.00,-1.62,-13.98,71.18,47.26,172138.0,-39288.0,241835.0,17.37,37.0,H-SC,8.17,174.0,-0.23,1.53,29.57,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,56.0,M-SC,10.15,220.0,-0.01,0.92,9.11,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,56.0,M-SC,10.15,220.0,-0.01,0.92,9.11,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.64,-13.20,15.23,0.02,23076.0,-23043.0,151516.0,131.03,58.0,M-SC,7.51,240.0,-1.00,0.96,31.79,OX40N,NTT,PAINTS
75,TATAELXSI,9161.00,9.50,-15.65,56.52,32.03,62854.0,-20628.0,111207.0,-8.65,73.0,H-SC,6.99,157.0,-0.33,0.70,24.56,OX40N,NTT,IT
80,TTKPRESTIG,770.00,-1.17,-20.60,26.06,0.09,20852.0,-20762.0,80015.0,85.70,38.0,M-SC,3.20,253.0,-1.00,0.51,5.12,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.99,-20.62,42.94,13.47,91923.0,-55593.0,214074.0,-67.34,55.0,H-SC,5.02,159.0,-0.60,1.35,8.85,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.63,4.29,15.77,20.74,33415.0,8718.0,211888.0,38.17,75.0,M-MC,6.29,189.0,0.26,1.34,15.36,XY25,NTT,FINANCE
17,COALINDIA,484.83,1.08,6.15,12.32,19.23,19676.0,9250.0,159710.0,24.27,84.0,L-LC,6.31,182.0,0.47,1.01,28.73,XY25,ATH,MINING
87,WIPRO,420.00,1.96,11.07,55.10,72.27,103702.0,18758.0,188206.0,-4.72,63.0,M-LC,6.08,101.0,0.18,1.19,17.93,XR,NTT,IT
49,MASFIN,398.61,-0.87,2.62,18.93,22.05,19033.0,2565.0,100545.0,-11.47,61.0,H-SC,6.92,168.0,0.13,0.63,46.15,XR,ATH,FINANCE
37,IEX,219.00,4.11,5.04,41.49,48.63,87144.0,10082.0,210036.0,-30.08,71.0,H-SC,13.83,137.0,0.12,1.33,17.25,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,1.96,11.07,55.10,72.27,103702.0,18758.0,188206.0,-4.72,63.0,M-LC,6.08,101.0,0.18,1.19,17.93,XR,NTT,IT
49,MASFIN,398.61,-0.87,2.62,18.93,22.05,19033.0,2565.0,100545.0,-11.47,61.0,H-SC,6.92,168.0,0.13,0.63,46.15,XR,ATH,FINANCE
37,IEX,219.00,4.11,5.04,41.49,48.63,87144.0,10082.0,210036.0,-30.08,71.0,H-SC,13.83,137.0,0.12,1.33,17.25,XR,NTT,MISC
25,FINCABLES,1641.55,0.08,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,56.0,M-SC,10.15,220.0,-0.01,0.92,9.11,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.12,-6.97,122.21,106.72,140224.0,-8596.0,114740.0,-53.87,37.0,H-SC,7.02,138.0,-0.06,0.72,14.78,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.35,-16.36,32.45,10.78,65223.0,-39322.0,200996.0,-50.14,13.0,X-LC,5.01,1.0,-0.60,1.27,0.00,X40,NTT,FMCG
7,AWL,485.00,-0.76,-25.77,111.75,57.19,270541.0,-84032.0,242095.0,-64.91,28.0,X-MC,8.44,58.0,-0.31,1.53,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,0.38,-18.39,60.85,31.26,114593.0,-42448.0,188320.0,-61.93,29.0,X-MC,18.17,56.0,-0.37,1.19,0.38,X40N,ATH,IT
52,PAGEIND,51605.07,0.06,-10.45,45.86,30.62,97352.0,-24774.0,212280.0,-36.35,33.0,X-MC,16.01,57.0,-0.25,1.34,0.06,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.05,-39.63,123.94,35.20,96282.0,-50986.0,77684.0,1.81,36.0,X-SC,17.26,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.00,0.09,1.21,19.32,20.77,45785.0,2838.0,236982.0,-7.61,39.0,X-MC,0.98,70.0,0.06,1.50,8.21,X40N,NTT,BREWERIES
68,SIEMENS,4671.50,0.18,-15.81,49.08,25.51,76899.0,-29415.0,156680.0,-3.93,50.0,H-LC,1.41,50.0,-0.38,0.99,15.46,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.11,1.84,41.10,43.70,103833.0,4568.0,252636.0,-6.50,66.0,X-MC,2.01,73.0,0.04,1.60,17.17,XY24,BTT,FMCG
84,VBL,671.64,2.18,2.96,31.77,35.67,103313.0,9347.0,325189.0,-9.20,71.0,X-LC,2.10,5.0,0.09,2.05,17.22,X40N,ATH,FMCG
45,JIOFIN,387.00,1.66,-2.38,27.51,24.48,71804.0,-6355.0,261010.0,-4.01,56.0,X-LC,2.28,37.0,-0.09,1.65,51.29,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.76,-25.77,111.75,57.19,270541.0,-84032.0,242095.0,-64.91,28.0,X-MC,8.44,58.0,-0.31,1.53,0.00,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.05,-39.63,123.94,35.20,96282.0,-50986.0,77684.0,1.81,36.0,X-SC,17.26,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
43,ITC,452.00,-0.35,-16.36,32.45,10.78,65223.0,-39322.0,200996.0,-50.14,13.0,X-LC,5.01,1.0,-0.60,1.27,0.00,X40,NTT,FMCG
52,PAGEIND,51605.07,0.06,-10.45,45.86,30.62,97352.0,-24774.0,212280.0,-36.35,33.0,X-MC,16.01,57.0,-0.25,1.34,0.06,X40,ATH,APPARELS
22,DIXON,18931.72,0.38,-18.39,60.85,31.26,114593.0,-42448.0,188320.0,-61.93,29.0,X-MC,18.17,56.0,-0.37,1.19,0.38,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,1.08,-25.53,91.82,42.84,44448.0,-16598.0,48408.0,-51.52,59.0,X-SC,28.50,83.0,-0.37,0.31,9.52,XY24,NTT,MISC
58,RELAXO,1176.00,-0.61,-47.95,192.61,52.31,145530.0,-69603.0,75557.0,-44.54,43.0,X-SC,7.60,92.0,-0.48,0.48,1.14,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.05,-39.63,123.94,35.20,96282.0,-50986.0,77684.0,1.81,36.0,X-SC,17.26,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,2.45,3.41,19.98,24.08,26230.0,4331.0,131281.0,-5.01,60.0,X-SC,6.71,91.0,0.17,0.83,23.70,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.22,-16.25,71.14,43.32,97035.0,-26472.0,136400.0,-28.06,52.0,X-SC,7.78,90.0,-0.27,0.86,4.73,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.35,-16.36,32.45,10.78,65223.0,-39322.0,200996.0,-50.14,13.0,X-LC,5.01,1.0,-0.60,1.27,0.00,X40,NTT,FMCG
78,TMPV,600.00,-1.50,-25.53,65.13,22.98,109569.0,-57667.0,168231.0,-23.08,51.0,X-LC,4.71,2.0,-0.53,1.06,5.81,XY24,NTT,AUTO
76,TCS,4389.97,1.22,-9.38,33.21,20.71,113825.0,-35495.0,342742.0,-22.01,60.0,X-LC,2.68,3.0,-0.31,2.16,14.52,X40,ATH,IT
84,VBL,671.64,2.18,2.96,31.77,35.67,103313.0,9347.0,325189.0,-9.20,71.0,X-LC,2.10,5.0,0.09,2.05,17.22,X40N,ATH,FMCG
41,INFY,1972.00,1.66,11.58,20.32,34.24,70939.0,36219.0,349107.0,-13.14,56.0,X-LC,7.56,6.0,0.51,2.20,20.82,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.79,-33.71,100.48,32.89,51423.0,-26030.0,51177.0,-732.12,64.0,L-MC,7.46,259.0,-0.51,0.32,40.94,XR,NTT,BANKS
49,MASFIN,398.61,-0.87,2.62,18.93,22.05,19033.0,2565.0,100545.0,-11.47,61.0,H-SC,6.92,168.0,0.13,0.63,46.15,XR,ATH,FINANCE
13,BSOFT,831.70,5.25,-19.61,87.57,50.79,99013.0,-27586.0,113067.0,3.21,59.0,H-SC,10.66,171.0,-0.28,0.71,31.65,XR,ATH,IT
72,SURYODAY,216.00,-0.05,-17.85,49.77,23.03,73214.0,-31967.0,147104.0,60.09,58.0,H-SC,5.22,165.0,-0.44,0.93,45.96,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,2.64,-13.20,15.23,0.02,23076.0,-23043.0,151516.0,131.03,58.0,M-SC,7.51,240.0,-1.00,0.96,31.79,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.82,-26.75,54.13,12.90,86638.0,-58440.0,160056.0,135.58,71.0,H-SC,10.59,173.0,-0.67,1.01,33.09,OX40N,NTT,FINANCE
75,TATAELXSI,9161.00,9.50,-15.65,56.52,32.03,62854.0,-20628.0,111207.0,-8.65,73.0,H-SC,6.99,157.0,-0.33,0.70,24.56,OX40N,NTT,IT
17,COALINDIA,484.83,1.08,6.15,12.32,19.23,19676.0,9250.0,159710.0,24.27,84.0,L-LC,6.31,182.0,0.47,1.01,28.73,XY25,ATH,MINING
85,VOLTAS,1856.00,1.44,17.94,23.11,45.20,52261.0,34398.0,226140.0,6.38,73.0,X-MC,4.14,80.0,0.66,1.43,25.96,XY25,NTT,AC
56,RAJOOENG,143.10,-3.45,-26.81,90.75,39.61,68081.0,-27480.0,75020.0,-50.46,62.0,H-SC,31.61,136.0,-0.40,0.47,21.43,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,21.79
1,25,44.79
2,50,75.52


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.21
LC    32.69
MC    28.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     24.14
X40      21.70
XY25     18.35
X40N     11.69
XR        8.35
AR        7.91
OX40N     6.97
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    29.50
X-MC    22.13
H-SC    21.01
M-SC    10.28
X-SC     7.20
H-MC     4.29
M-MC     1.34
M-LC     1.19
L-LC     1.01
H-LC     0.99
L-SC     0.72
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.50,-7.34,42.56
IT,11.72,-15.05,73.09
FINANCE,10.20,-10.35,56.67
REFINERIES,9.80,-3.57,29.77
MISC,6.60,-24.33,75.19
ELECTRICAL,5.44,-8.93,48.83
PAINTS,5.22,-11.80,28.40
INSURANCE,4.26,-0.71,35.18
PHARMA,3.61,-2.62,34.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3151334.0,21
AR,1317400.0,10
XR,1168633.0,12
X40,1081382.0,15
X40N,891655.0,9
XY25,799322.0,10
OX40N,687246.0,10
SR,283184.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3465677.0,24
X-MC,1529552.0,16
M-SC,1430027.0,15
X-LC,1312253.0,16
X-SC,781250.0,8
H-MC,400089.0,3
L-SC,176193.0,2
M-LC,103702.0,1
H-LC,76899.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1231341.0      6
           AR         907684.0      5
M-SC       XY24       821855.0      6
H-SC       XR         745681.0      7
X-LC       XY25       528943.0      6
X-MC       X40        486547.0      7
           XY24       436682.0      3
X-LC       X40        405642.0      6
X-MC       X40N       389035.0      4
X-SC       X40N       306325.0      3
M-SC       OX40N      298785.0      5
H-SC       OX40N      297787.0      4
X-SC       XY24       285732.0      3
H-SC       SR         283184.0      2
X-MC       XY25       217288.0      2
H-MC       AR         205738.0      2
X-LC       X40N       196295.0      2
H-MC       XY24       194351.0      1
X-SC       X40        189193.0      2
M-SC       XR         182308.0      2
X-LC       XY24       181373.0      2
M-SC       AR         127079.0      2
M-LC       XR         103702.0      1
L-SC       OX40N       90674.0      1
           XR          85519.0      1
H-LC       AR          76899.0      1
L-MC       XR          51423.0      1
M-MC       XY25        33415.0      1
L-LC       XY25        19676.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
